In [36]:
from langchain_openai import ChatOpenAI
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_milvus import Milvus
from dotenv import load_dotenv

load_dotenv()
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
llm = ChatOpenAI(model="doubao-seed-1-6-lite-251015")

vectorstore_loaded = Milvus(
    embeddings,
    connection_args={"host": "localhost", "port": "19530"},
    collection_name="huozhe_collection",
)

retriever = vectorstore_loaded.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 10, "score_threshold": 0.3}  # 检索最相似的10个文档，默认L2距离，越小越相似
)

In [37]:
from langchain_core.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
    retriever=retriever,
    name="huozhe_tool",
    description="一个关于作家余华的小说《活着》，其中包含了这本小说的全部内容。",
)

tools = [retriever_tool]

In [38]:
from typing import TypedDict, Annotated, Sequence, Literal
from langchain_core.messages import BaseMessage
from langgraph.graph import add_messages
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

In [39]:
from langchain_core.messages import HumanMessage
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode

def agent(state):
    messages = state["messages"]
    model = llm.bind_tools(tools)
    response = model.invoke(messages)
    return {"messages": [response]}


def should_continue(state):
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        return "tools"
    return END

tool_node = ToolNode(tools)
graph = StateGraph(AgentState)

graph.add_node("agent", agent)
graph.add_node("tools", tool_node)

graph.set_entry_point("agent")
graph.add_conditional_edges("agent", should_continue, {
    "tools": "tools",
    END: END
})
graph.add_edge("tools", "agent")

app = graph.compile()

In [40]:
result = app.invoke({
    "messages": [
        HumanMessage(content="请告诉我活着中有庆是怎么死的？")
]})["messages"][-1].content

print(result)

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('422 Client Error: unknown for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unprocessable entity: invalid JSON part for post.aff2ca87-30e5-49da-aa4c-aa31a9e8f0e8: could not unmarshal run: unexpected end of JSON input"}\n')
No index params provided. Could not determine relevance function. Use L2 distance as default.


在《活着》中，**有庆是因给县长的妻子（学校校长）献血而死**，具体情节如下：


### 死亡背景
有庆念五年级时，学校校长（县长的女人）在医院生孩子大出血，生命垂危。学校老师紧急集合五年级学生去医院献血，孩子们因是给“校长献血”而积极响应。


### 死亡过程
1. **有庆的表现**：有庆第一个跑到医院，却因“不遵守纪律”被老师训斥，只能站在一旁看其他同学验血。当十多个同学血型都不对时，有庆主动说“我献过血”（之前为生病的母亲家珍献过），最终血型匹配。
2. **致命的抽血**：医生让护士给有庆抽血，有庆因身体瘦弱（长期营养不良），被**过量抽血**。他抽血时感到头晕，闭眼后再也没醒来，当场死亡。


### 悲剧本质
有庆的死是时代悲剧的缩影：他的善良和积极（想为校长献血），却因医疗条件简陋、医生的疏忽（或冷漠），最终成为牺牲品。这也让福贵的人生再次陷入绝望，失去了唯一的儿子。


（注：该情节出自小说《活着》，反映了底层小人物在时代洪流中的无力与苦难。）
